# Testing visualisations and pipelines


> Here we want to get started on connecting the CKAN API and, where available, visualise the data. 

Where the API does not have any data available, dummy data will be produced to replicate the KPIs of each pilot city. It is recognised that the published data may look somewhat different, but the aim of this task is to get started on the pipeline

> list of KPIs 
* KPI#1 - Increases in pedestrians + cyclists (numbers)
* KPI#2 -Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI#3 -Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI#4 - VRUs/user interactions improvements (AI modelled - reduced near misses, responsive traffic lights, etc.)
* KPI#5 - Reallocation of public space (sqm/year)
* KPI#6 - Conversion from impermeable to permeable/vegetated surface (sqm + no. of trees planted)
* KPI#7 - Uptake/incorporation of Circular Economy principles (% from baseline)
* KPI#8 - Engagement (no. of people engaged in co-creation/co-management)
* KPI#9 - Public acceptance of interventions (citizens + stakeholders) (% of persons asked)
* KPI#10 - Extendibility - Replicability of the intervention (% extendable/ replicable)
* KPI#11- Promotion of infection free mobility in Interventions (possibility to keep 1-2m distance)
* KPI#12 - Achievement of Climate Targets for Transport (% of achievement)

## Connect to the API

In [87]:
import requests

CKAN_URL = 'https://reallocate-ckan.iti.gr'
API_KEY = ''  # Replace if needed

headers = {'Authorization': API_KEY}

# Get all organization names
orgs_url = f'{CKAN_URL}/api/3/action/organization_list'
orgs_response = requests.get(orgs_url, headers=headers)
org_names = orgs_response.json()['result']
print(org_names)

['bkk', 'bsc', 'barcelona', 'bologna', 'budapest', 'certh', 'cerema', 'dekra', 'ertico', 'gothenburg', 'heidelberg', 'lyon', 'tampere', 'ucd', 'utrecht', 'warsaw', 'zagreb']


In [88]:
from pprint import pprint

all_data = {}

for org_name in org_names:
    search_url = f'{CKAN_URL}/api/3/action/package_search'
    params = {'fq': f'organization:{org_name}', 'rows': 100}
    res = requests.get(search_url, headers=headers, params=params)
    result = res.json()

    if result['success']:
        datasets = result['result']['results']
        all_data[org_name] = []

        for dataset in datasets:
            dataset_info = {
                'title': dataset['title'],
                'name': dataset['name'],
                'resources': []
            }

            for res in dataset.get('resources', []):
                resource_info = {
                    'name': res['name'],
                    'format': res['format'],
                    'url': res['url']
                }
                dataset_info['resources'].append(resource_info)

            all_data[org_name].append(dataset_info)
    else:
        print(f"Failed to fetch datasets for org: {org_name}")


In [89]:
all_data.items()

dict_items([('bkk', [{'title': 'Megyeri Road school modal split study', 'name': 'megyeri-road-school-modal-split-study', 'resources': [{'name': 'Megyeri road school modal split data', 'format': 'PPT', 'url': 'https://reallocate-ckan.iti.gr:443/dataset/776f6a6b-e358-4cdc-81b6-e22a953a7f5f/resource/42b6c808-ac87-45e8-867c-afd62b7d913a/download/sulizona_megyeri-uti-kiertekeles_eng.pptx'}]}, {'title': 'EFM Unified traffic model', 'name': 'efm-unified-traffic-model', 'resources': [{'name': 'EFM-Unified traffic model', 'format': 'JPEG', 'url': 'https://reallocate-ckan.iti.gr:443/dataset/b09d25cc-0e49-4fa6-ad1f-fec4bbe0df44/resource/8e22de31-8ec7-482c-978f-04729a8d9e4e/download/efm-20241114t161226z-001.zip'}]}, {'title': 'Hospital data', 'name': 'hospital-data', 'resources': [{'name': 'Hospital_data.xlsx', 'format': 'XLSX', 'url': 'https://reallocate-ckan.iti.gr:443/dataset/e197ecca-a1ff-4830-8220-02b44018a36c/resource/946f65aa-3a16-4109-aa99-58f4cc46707a/download/hospital_data.xlsx'}]}, {'ti

In [90]:
import pandas as pd

for org, datasets in all_data.items():
    print(f"\n📁 Organization: {org}")
    for ds in datasets:
        print(f"  📄 Dataset: {ds['title']}")
        for res in ds['resources']:
            print(f"    🔗 Resource: {res['name']} ({res['format']})")
            if res['format'].lower() == 'csv':
                try:
                    df = pd.read_csv(res['url'])
                    print(f"      ✅ Loaded {len(df)} rows")
                    # You can now store/save/analyze `df`
                except Exception as e:
                    print(f"      ❌ Failed to read CSV: {e}")



📁 Organization: bkk
  📄 Dataset: Megyeri Road school modal split study
    🔗 Resource: Megyeri road school modal split data (PPT)
  📄 Dataset: EFM Unified traffic model
    🔗 Resource: EFM-Unified traffic model (JPEG)
  📄 Dataset: Hospital data
    🔗 Resource: Hospital_data.xlsx (XLSX)
  📄 Dataset: Baross road traffic data
    🔗 Resource: Baross road traffic data (PPT)

📁 Organization: bsc

📁 Organization: barcelona
  📄 Dataset: Bicing service use of the city of Barcelona from August 2018 to March 2019
    🔗 Resource: us-del-servei-bicing ()
  📄 Dataset: Vehicles involved in accidents handled by the police in the city of Barcelona
    🔗 Resource: 2023_accidents_vehicles_gu_bcn.csv (CSV)
      ✅ Loaded 14361 rows
  📄 Dataset: Traffic state information by sections of the city of Barcelona
    🔗 Resource: ec409c44-6a07-411d-8f53-efb5c1b7e989 ()
  📄 Dataset: Survey of Municipal Services 2024
    🔗 Resource: Survey of Municipal Services 2024 (CSV)
      ✅ Loaded 6000 rows

📁 Organization: 

In [91]:
df

,sequence_id,municipality,location,location_type,counting_method,date,morning_peak_hour_cyclists,morning_peak_hour_pedestrians,morning_peak_hour_total,evening_peak_hour_cyclists,evening_peak_hour_pedestrians,evening_peak_hour_total,daily_total_cyclists,daily_total_pedestrians,daily_total_total,speed_limit,average_speed_pedestrians,percentile_85_speed_pedestrians,result_expired,note
0,293.0,837.0,Mäyränmäenkatu,Pent,Video calculation,2018-06-12z,NaN,NaN,NaN,6.0,37.0,43.0,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
1,295.0,837.0,Mäyränmäenkatu,Carriageway,Video calculation,2018-06-12z,NaN,NaN,NaN,0.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
2,296.0,837.0,Vuores Park Street,Pent,Video calculation,2018-06-12z,NaN,NaN,NaN,24.0,13.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
3,297.0,837.0,Vuores Park Street,Carriageway,Video calculation,2018-06-12z,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
4,298.0,837.0,Vuores Park Street,Pent,Video calculation,2018-06-12z,NaN,NaN,NaN,18.0,18.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,7933.0,837.0,Vuores Park Street,Zebra crossing,Ecocounter to move,2024-10-23z,NaN,NaN,37.0,NaN,NaN,49.0,NaN,NaN,513.0,NaN,NaN,NaN,No,NaN
148,7934.0,837.0,Vuores Park Street,Zebra crossing,Ecocounter to move,2024-10-22z,NaN,NaN,10.0,NaN,NaN,26.0,NaN,NaN,156.0,NaN,NaN,NaN,Yes,NaN
149,7935.0,837.0,Vuores Park Street,Zebra crossing,Ecocounter to move,2024-10-23z,NaN,NaN,6.0,NaN,NaN,24.0,NaN,NaN,171.0,NaN,NaN,NaN,No,NaN
150,7936.0,837.0,Vuores Park Street,Zebra crossing,Ecocounter to move,2024-10-22z,NaN,NaN,7.0,NaN,NaN,33.0,NaN,NaN,163.0,NaN,NaN,NaN,Yes,NaN


## Pilot cities

### Barcelona

#### Pilot 1 - Pedestrians, cyclists & micro-mobility vehicles in shared spaces

**KPIs** 
* KPI #1 - Increases in pedestrians + cyclists (numbers)
* KPI #2 - Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI #3 - Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI #5 - Reallocation of public space (sqm/year)
* KPI #6 - Conversion from impermeable to permeable/vegetated surface (sqm + no. of trees planted)
* KPI #10 - Extendibility - Replicability of the intervention (% extendable/ replicable)

In [92]:
import pandas as pd

org = 'barcelona'
datasets = all_data.get(org, [])
for ds in datasets:
    print(f"Dataset: {ds['title']}")
    for res in ds['resources']:
        print(f"    🔗 Resource: {res['name']} ({res['format']})")
        print(f"      URL: {res['url']}")
    


Dataset: Bicing service use of the city of Barcelona from August 2018 to March 2019
    🔗 Resource: us-del-servei-bicing ()
      URL: https://opendata-ajuntament.barcelona.cat/data/en/dataset/us-del-servei-bicing
Dataset: Vehicles involved in accidents handled by the police in the city of Barcelona
    🔗 Resource: 2023_accidents_vehicles_gu_bcn.csv (CSV)
      URL: https://reallocate-ckan.iti.gr:443/dataset/8ffbf107-042b-41c9-9a6e-a7119bce5d63/resource/8e0da597-8a35-4cd4-b35e-b46b0033a748/download/2023_accidents_vehicles_gu_bcn.csv
Dataset: Traffic state information by sections of the city of Barcelona
    🔗 Resource: ec409c44-6a07-411d-8f53-efb5c1b7e989 ()
      URL: https://opendata-ajuntament.barcelona.cat/data/en/dataset/trams/resource/ec409c44-6a07-411d-8f53-efb5c1b7e989
Dataset: Survey of Municipal Services 2024
    🔗 Resource: Survey of Municipal Services 2024 (CSV)
      URL: https://reallocate-ckan.iti.gr:443/dataset/10c6b922-3f58-4bef-ae42-b45e9bb19fd4/resource/d609c0d6-1b34

so for now it looks like we have two csv's available, and two links that take us to the OpendataBarcelona site

#### Pilot 2 - Increased & integrated public transport accessibility for people with disabilities

**KPIs** 
* KPI #1 - Increases in pedestrians + cyclists (numbers)
* KPI #2 - Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI #3 - Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI #4 - VRUs/user interactions improvements (AI modelled - reduced near misses, responsive traffic lights, etc.)
* KPI #5 - Reallocation of public space (sqm/year)
* KPI #10 - Extendibility - Replicability of the intervention (% extendable/ replicable)
* KPI #12 - Achievement of Climate Targets for Transport (% of achievement)

### Bologna

#### Pilot - safe & sustainable school district along the Knowledge Path

**KPIs** 
* KPI #1 - Increases in pedestrians + cyclists (numbers)
* KPI #2 - Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI #3 - Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI #5 - Reallocation of public space (sqm/year)
* KPI #6 - Conversion from impermeable to permeable/vegetated surface (sqm + no. of trees planted)
* KPI #7 - Uptake/incorporation of Circular Economy principles (% from baseline)
* KPI #10 - Extendibility - Replicability of the intervention (% extendable/ replicable)
* KPI #12 - Achievement of Climate Targets for Transport (% of achievement)

### Budapest

#### Pilot 1 - Introducing 'Healthy Superblocks

* KPI#1 - Increases in pedestrians + cyclists (numbers)
* KPI#2 -Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI#3 -Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI#5 - Reallocation of public space (sqm/year)
* KPI#6 - Conversion from impermeable to permeable/vegetated surface (sqm + no. of trees planted)
* KPI#10 - Extendibility - Replicability of the intervention (% extendable/ replicable)


#### Pilot 2 - Peri-urban traffic safety

* KPI#1 - Increases in pedestrians + cyclists (numbers)
* KPI#2 -Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI#3 -Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI#4 - VRUs/user interactions improvements (AI modelled - reduced near misses, responsive traffic lights, etc.)
* KPI#5 - Reallocation of public space (sqm/year)
* KPI#10 - Extendibility - Replicability of the intervention (% extendable/ replicable)
* KPI#12 - Achievement of Climate Targets for Transport (% of achievement)

### Gothenburg

#### Pilot 1 - Safe System Approach for children’s active travel in peri-urban areas

* KPI#4 - VRUs/user interactions improvements (AI modelled - reduced near misses, responsive traffic lights, etc.)
* KPI#10 - Extendibility - Replicability of the intervention (% extendable/ replicable)
* KPI#12 - Achievement of Climate Targets for Transport (% of achievement)

#### Pilot 2 - Seamless travel, citizen engagement & nudging tools in a complex mobility hub

* KPI#1 - Increases in pedestrians + cyclists (numbers)
* KPI#2 -Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI#3 -Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI#4 - VRUs/user interactions improvements (AI modelled - reduced near misses, responsive traffic lights, etc.)
* KPI#9 - Public acceptance of interventions (citizens + stakeholders) (% of persons asked)


### Heidelberg

#### Pilot 1 - Regional commuter plans with mobility hubs

* KPI#9 - Public acceptance of interventions (citizens + stakeholders) (% of persons asked)
* KPI#10 - Extendibility - Replicability of the intervention (% extendable/ replicable)
* KPI#12 - Achievement of Climate Targets for Transport (% of achievement)

#### Pilot 2 - Contextual & tactical public space reallocation

* KPI#1 - Increases in pedestrians + cyclists (numbers)
* KPI#2 -Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI#3 -Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI#4 - VRUs/user interactions improvements (AI modelled - reduced near misses, responsive traffic lights, etc.)
* KPI#5 - Reallocation of public space (sqm/year)
* KPI#6 - Conversion from impermeable to permeable/vegetated surface (sqm + no. of trees planted)
* KPI#7 - Uptake/incorporation of Circular Economy principles (% from baseline)
* KPI#9 - Public acceptance of interventions (citizens + stakeholders) (% of persons asked)
* KPI#10 - Extendibility - Replicability of the intervention (% extendable/ replicable)
* KPI#12 - Achievement of Climate Targets for Transport (% of achievement)

### Lyon

#### Pilot 1 - Safety measures for 'Vision-Zero' in the school areas

* KPI#1 - Increases in pedestrians + cyclists (numbers)
* KPI#2 -Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI#3 -Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI#4 - VRUs/user interactions improvements (AI modelled - reduced near misses, responsive traffic lights, etc.)
* KPI#5 - Reallocation of public space (sqm/year)
* KPI#6 - Conversion from impermeable to permeable/vegetated surface (sqm + no. of trees planted)
* KPI#10 - Extendibility - Replicability of the intervention (% extendable/ replicable)


#### Pilot 2 - Road safety technology & non-polluting parking policy

* KPI#9 - Public acceptance of interventions (citizens + stakeholders) (% of persons asked)
* KPI#12 - Achievement of Climate Targets for Transport (% of achievement)

### Tampere

#### Pilot - AI for increased road safety, space reallocation & parametric design

* KPI#1 - Increases in pedestrians + cyclists (numbers)
* KPI#2 -Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI#3 -Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI#4 - VRUs/user interactions improvements (AI modelled - reduced near misses, responsive traffic lights, etc.)
* KPI#5 - Reallocation of public space (sqm/year)
* KPI#9 - Public acceptance of interventions (citizens + stakeholders) (% of persons asked)
* KPI#10 - Extendibility - Replicability of the intervention (% extendable/ replicable)


### Utrecht

#### Pilot - Safety-proofing schools in vulnerable neighbourhoods

* KPI#1 - Increases in pedestrians + cyclists (numbers)
* KPI#2 -Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI#3 -Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI#4 - VRUs/user interactions improvements (AI modelled - reduced near misses, responsive traffic lights, etc.)
* KPI#5 - Reallocation of public space (sqm/year)
* KPI#6 - Conversion from impermeable to permeable/vegetated surface (sqm + no. of trees planted)
* KPI#8 - Engagement (no. of people engaged in co-creation/co-management)
* KPI#9 - Public acceptance of interventions (citizens + stakeholders) (% of persons asked)
* KPI#10 - Extendibility - Replicability of the intervention (% extendable/ replicable)


### Warsaw

#### Pilot - Warsaw’s green & safe road to school

* KPI#1 - Increases in pedestrians + cyclists (numbers)
* KPI#2 -Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI#3 -Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI#4 - VRUs/user interactions improvements (AI modelled - reduced near misses, responsive traffic lights, etc.)
* KPI#5 - Reallocation of public space (sqm/year)
* KPI#6 - Conversion from impermeable to permeable/vegetated surface (sqm + no. of trees planted)
* KPI#7 - Uptake/incorporation of Circular Economy principles (% from baseline)
* KPI#8 - Engagement (no. of people engaged in co-creation/co-management)
* KPI#9 - Public acceptance of interventions (citizens + stakeholders) (% of persons asked)
* KPI#10 - Extendibility - Replicability of the intervention (% extendable/ replicable)
* KPI#12 - Achievement of Climate Targets for Transport (% of achievement)

### Zagreb

#### Pilot - holistic solutions for the Central traffic corridor

* KPI#1 - Increases in pedestrians + cyclists (numbers)
* KPI#2 -Pedestrian & disabled comfort (reduced risks; walking dstance/time; shade, walkable conditions) (% from baseline)
* KPI#3 -Cycling & e-bike comfort (reduced cycle waiting time, increased bike parkings, etc.) (% from baseline)
* KPI#5 - Reallocation of public space (sqm/year)
* KPI#6 - Conversion from impermeable to permeable/vegetated surface (sqm + no. of trees planted)
* KPI#7 - Uptake/incorporation of Circular Economy principles (% from baseline)
* KPI#10 - Extendibility - Replicability of the intervention (% extendable/ replicable)
* KPI#12 - Achievement of Climate Targets for Transport (% of achievement)

### BKK (Budapest)
### BSC
### CERTH
### Cerema
### DEKRA
### Ertico
### UCD



fetch_ckan_data(API_KEY, BASE_URL)

In [25]:
response_dict

{'help': 'https://reallocate-ckan.iti.gr:443/api/3/action/help_show?name=group_list',
 'success': True,
 'result': ['barcelona-pilot-1',
  'barcelona-pilot-2',
  'budapest-pilot-1',
  'budapest-pilot-2',
  'certh-testing',
  'gothenburg-pilot-1',
  'gothenburg-pilot-2',
  'heidelberg-pilot-1',
  'heidelberg-pilot-2',
  'lyon-pilot-1',
  'lyon-pilot-2',
  'reallocate']}